In [2]:
#Imports
import numpy as np
import tsplib95
import networkx

import warnings
warnings.filterwarnings("ignore")

#np.seterr(all='raise')

import random
import pandas as pd
from scipy.spatial import distance


# Ant Colony Optimisation Algorithm

## <center> Probability matrix </center>
Probability between vertices is given by:
$$\begin{equation*} P_{ij} = \dfrac{(\tau_{ij})^\alpha (n(c_{ij}))^\beta}{\sum_{j=0}^{m} (\tau_{ij})^\alpha (n(c_{ij}))^\beta }  \end{equation*} $$


where $c_{ij}$ is the length of the edge 

$\tau_{ij}$ is the pheromone count on the edge $c_{ij}$

$n(c_{ij}) = \dfrac{1}{c_{ij}}$

can be interpreted as the Probability of going to node j from i

In [3]:
def getProbabilityMatrix(A,P,alpha,beta):
    """
        Generates the probability matrix based on the pheromone and distance matrix

        Inputs
            A     : the distance matrix
            P     : the Pheromone Matrix
            alpha : the Pheromone weight value
            beta  : the distance weight value

        Outputs
            X     : the probability matrix
    """

    ########METHOD 1##########################
    # C = 1/A
    # X = (P**alpha) * (C ** beta)

    # n = len(X)

    # for i in range(n):
    #     deno = sum(X[i,:])
    #     X[i,:] = (1/deno) * X[i,:]
    
    # return X
    ##########################################

    #####METHOD 2#############################
    n = len(A)

    X = np.zeros((n,n))

    for i in range(0,n,1):

        #calculating the total weight across path
        deno = 0

        for j in range(0,n,1):

            if(A[i,j]!=0):
                deno += (P[i,j]**alpha)*((1/A[i,j])**beta)

        #updating probability matrix
        for j in range(0,n,1):
            
            if i!=j and A[i,j]!=0 and A[i,j]!=np.inf:
                X[i,j] = (P[i,j]**alpha)*((1/A[i,j])**beta)/deno
            else:
                X[i,j] = 0
    
    return X
    ##############################################################


In [4]:
#TESTING#
A = np.array([[np.inf,5,2,200],[5,np.inf,100,4],[2,100,np.inf,3],[200,4,3,np.inf]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = getProbabilityMatrix(A, P, alpha, beta)
Prob

array([[0.        , 0.28368794, 0.70921986, 0.0070922 ],
       [0.43478261, 0.        , 0.02173913, 0.54347826],
       [0.59288538, 0.01185771, 0.        , 0.39525692],
       [0.00849858, 0.42492918, 0.56657224, 0.        ]])

## <center> Accumulator</center>

generates the Accumulator vector. which is basically the sum of itself and all the probabilities to the right of it

for example given the probabilities $v = [0.76,0.19,0.05]$ 

the Accumulative vector will be $u = [1,0.24,0.05]$

after this a random number will be generated. $r \in [0,1]$

then the function will return the index in which r is inbetween the vector u

for example if $ 0.24 \leq r \leq 1 $ the function will return 0

In [5]:
def accumulator(v):
    """
        Creates the accumulator vector from the input given
    """

    u = []

    for i in range(len(v)):
        temp = sum(v[i:])
        u.append(temp)

    r = random.uniform(0,u[0])
    index = -1

    for i in range(0,len(u),1):
        
        if i!=len(u)-1:
            if (r<=u[i]) and (r>u[i+1]):
                index = i
                break
        else:
            if (r>0) and (r<=u[-1]):
                index = i

    return index

In [6]:
#TESTING
#v = [0.76,0.19,0.05]
v = [0,0.28368794,0.70921986,0.0070922]
accumulator(v)

2

In [7]:
def ant_path(A,P,Q,start):
    """
        Inputs:
            A     : distance matrix
            P     : probability matrix
            Q     : pheromone update parameter
            start : index of starting location of agent

        Outputs:
            dist  : distance traveled by agent
            path  : path taken by agent

            C     : pheromone update trail
    """

    n = len(A)
    X = np.copy(P)
    C = np.zeros((n,n))
    dist = 0
    path = [start]
    i = start
    count = 0
    valid = True

    while count < n-1:

        #getting the row of probabilities corresponding current node#
        v = X[i,:]

        #getting next node to visit#
        j = accumulator(v)

        #checks if solution is valid#
        if(j==-1):
            valid = False
            break

        dist+= A[i,j]
        path.append(j)

        #making sure agent doesnt visit node twice#
        X[:,i] = 0

        i = j
        count+=1
    

    dist+= A[i,start]
    path.append(start)

    for j in range(len(path)-1):
        row = path[j]
        col = path[j+1]
        C[row,col] += Q/dist
    

    return dist,path,C,valid

In [8]:
#TESTING#
A = np.array([[np.inf,5,2,200],[5,np.inf,100,4],[2,100,np.inf,3],[200,4,3,np.inf]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = getProbabilityMatrix(A, P, alpha, beta)
dist,path,C,valid = ant_path(A,Prob,Q = 1,start = 0)

In [9]:
dist

14.0

In [10]:
path

[0, 1, 3, 2, 0]

In [11]:
C

array([[0.        , 0.07142857, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.07142857],
       [0.07142857, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.07142857, 0.        ]])

In [12]:
valid

True

In [13]:
def reorder_path(path,start):
    """
        reorders path starting and ending with start index

        Input:
            path     : path which is a permuatation that will be reordered
            start    : the starting index 

        outputs:
            new_path : reordered path
    """

    index = path.index(start)

    if index!=0:
        new_path = []

        for i in range(index,len(path),1):
            new_path.append(path[i])
        
        for i in range(1,index,1):
            new_path.append(path[i])
        
        new_path.append(start)
        return new_path
    else:
        return path



In [14]:
#TEST#
reorder_path(path,start=3)

[3, 2, 0, 1, 3]

## <center> Ant Colony Optimiser </center>



### <center> variable evaporation rate: </center>

$$\rho = 1 - \frac{i+1}{n}$$

where:


$i$ : current iteration


$n$ : number of iterations

In [15]:
def updatePheromone(p,X,C,i,n):
    """
        updates the pheromone matrix depending on parameters entered
    """

    s = p

    if p == -1:
        s = 1 - (i+1)/n
    
    ans = (1-s)*X + C
    
    return ans

In [16]:
def sort_elite(elite_dist,elite_C):
    """
        sorts the elite lists
    """
    n = len(elite_dist)
    dist = elite_dist.copy()
    C = elite_C.copy()
    
    for i in range(0,n,1):
        for j in range(i+1,n,1):
            
            if elite_dist[j]<elite_dist[i]:
                
                #sorting distances#
                temp_dist = dist[i]
                dist[i] = dist[j]
                dist[j] = temp_dist
                
                #sorting pheromone matrix#
                temp_C = C[i]
                C[i] = C[j]
                C[j] = temp_C
    
    return dist,C

In [17]:
def update_Elite(elite_dist,elite_C,ratio,k,temp_dist,temp_C):
    """
        updates the list of elite agents that will be used to update the trail for the following agents
    """
    
    size = round(ratio*k)
    dist = elite_dist.copy()
    C = elite_C.copy()
    
    dist.append(temp_dist)
    C.append(temp_C)
    
    new_dist,new_C = sort_elite(dist,C)
    
    while len(new_dist)>size:
        new_dist.pop()
        new_C.pop()
    
    return new_dist,new_C

In [ ]:
def get_elite_C(elite_C):
    """
        sums up the elite matrices into 1 matrix
    """
    n = len(elite_C)
    shape = len(elite_C[0])
    
    new_C = np.zeros((shape,shape))
    
    for i in range(n):
        new_C += elite_C[i]
    
    return new_C

In [ ]:
def ACO(A,p,alpha,beta,n,k,Q,random_loc,update,ratio,max_rep,tol,log):
    """
    Finds optimal route using Ant Colony Optimisation techniques
    
    Inputs:
        A           : Distance Matrix
        p           : (scalar) evaporation rate
        alpha       : (scalar) parameter that affects pheromone weighting
        beta        : (scalar) parameter that affects distance weighting
        n           : (scalar) number of interations to be performed
        k           : (scalar) number of ants to be used
        random_loc  : boolean variable that assigns agents to random nodes or not
        update      : how the pheromone matrix is updated (all: all agents pheromone is updated) ; (best: best agent updates pheromone matrix)
        ratio       : percentage of the top solution to select from
        max_rep     : how times same solutions pop up before terminating 
        tol         : the tolerance solutions generated
        
    Output:
        dist        : the distance traveled by the last agent
        path        : set of 2-tuples of route to be taken
    """

    #Pheromone Matrix#
    X = np.ones((len(A),len(A)))

    #intialisation#
    P = getProbabilityMatrix(A,X, alpha, beta)
    start = 0

    if random_loc == True:
        start = np.random.randint(len(A))


    valid = False

    a_rep = 0
    while valid == False and a_rep<max_rep:
        best_dist,best_path,best_C,valid = ant_path(A,P,Q,start)
        a_rep+=1
    
    if a_rep >=max_rep:
        print("Error")
        return 0,0

    rep = 0
    
    
    #elitism#
    elite_dist = [best_dist]
    elite_C = [best_C]

    for i in range(n):

        C_all = np.zeros((len(A),len(A)))

        #getting probability matrix#
        P = getProbabilityMatrix(A,X, alpha, beta)

        #constructing ant paths#
        valid = False
        
        a_rep = 0
        while valid == False and a_rep<max_rep:
            iter_dist,iter_path,iter_C,valid = ant_path(A,P,Q,start)
            a_rep+=1
        
        if a_rep >=max_rep:
            return best_dist,best_path
        

        for j in range(k):
            start = 0

            if random_loc == True:
                start = np.random.randint(len(A))
            
            dist_temp,path_temp,C_temp,valid = ant_path(A,P,Q,start)

            if update =='all' and valid == True:
                C_all+= C_temp
            elif update =='elite' and valid == True:
                elite_dist,elite_C = update_Elite(elite_dist,elite_C,ratio,k,dist_temp,C_temp)
            
            #getting best value per iteration#
            if dist_temp < iter_dist:
                iter_dist = dist_temp
                iter_path = path_temp
                iter_C = C_temp
        
        

        #handling repeating values#
        if(np.abs(iter_dist-best_dist)<=tol):
            rep+=1
        else:
            rep = 0

        #updating the best iteration#
        if iter_dist<=best_dist:
            best_dist = iter_dist
            best_path = iter_path
            best_C = iter_C
        

        if update == 'best':
            X = updatePheromone(p,X,best_C,i,n)
            #X = (1-p)*X + best_C
        elif update == 'all':
            X = updatePheromone(p,X,C_all,i,n)
            #X = (1-p)*X + C_all
        elif update == 'elite':
            C_new = get_elite_C(elite_C)
            X = updatePheromone(p,X,C_new,i,n)
            
        

        if log == True:
            print("================================== \nbest distance at iteration {} : {} \n current best :{}".format(i,iter_dist,best_dist))
        

        if rep == max_rep:
            break
    

    #reorder path#
    best_path = reorder_path(best_path,start=0)
    return best_dist,best_path



# Tools

In [ ]:
def get_distance_matrix_symmetric(path):

        """
            Creates a distance matrix based on the data proved in path

            Inputs:

                path : the path to the text file containing information on each node
            
            Outputs:
                A    : the distance matrix
        """
        
        with open(path) as reader :

            first_lines = 0
            i = 0
            X = []

            for lines in reader.readlines():

                if(first_lines>5 and lines!='EOF' and lines!='EOF\n'):
                    
                    stripped_line = lines.strip()
                    list_line = stripped_line.split()
                    w = [float(i) for i in list_line[1:]]
                    X.append(w)

                first_lines+=1
        
        X = np.array(X)
        m,n = X.shape

        A = []

        for i in range(0,m,1):
            u = X[i,:]
            dist = []

            for j in range(0,m,1):

                if i!=j:
                    v = X[j,:]
                    d = distance.euclidean(u,v)
                    dist.append(d)
                else:
                    dist.append(np.inf)

            A.append(dist)
        
        A = np.array(A)
        return A

In [ ]:
def get_distance_matrix_asymmetric(path):

        """
            Creates a distance matrix based on the data proved in path

            Inputs:

                path : the path to the text file containing information on each node
            
            Outputs:
                A    : the distance matrix
        """
        
        with open(path) as reader :

            first_lines = 0
            i = 0
            X = []

            for lines in reader.readlines():

                if(first_lines>6 and lines!='EOF' and lines!='EOF\n'):
                    
                    stripped_line = lines.strip()
                    list_line = stripped_line.split()
                    w = [float(i) for i in list_line[1:]]

                    if len(w)!=0:
                        X.append(w)

                first_lines+=1
        
        X = np.array(X)
        return X

### Random Walk

In [ ]:
def ACO_random_walk(A,x0,update_rule,alpha,size,N,rep,tol,log):
    
    n = len(x0)
    xnew = x0

    f_old = 0

    for _ in range(rep):
        old_dist,old_path = ACO(A,p=x0[0],alpha=x0[1],beta=x0[2],n = round(x0[3]),k=round(x0[4]),Q=x0[5],random_loc=True,update=update_rule,ratio = 0,max_rep=20,tol = 0.001,log=False)
        f_old+= old_dist
    
    f_old = f_old / rep

    
    while alpha>tol:
        i = 1
        
        while i<N:
            
            v = np.random.uniform(-size,size,n)
            a = 1/(np.linalg.norm(v))
            
            
            if a<=size:
                u = a*v
                xnew = x0 + alpha*u 

                f_new = 0

                for _ in range(rep):
                    new_dist,new_path = ACO(A,p=xnew[0],alpha=xnew[1],beta=xnew[2],n = round(xnew[3]),k=round(xnew[4]),Q=xnew[5],random_loc=True,update='best',ratio = 0,max_rep=20,tol = 0.001,log = False)
                    f_new+= new_dist
            
                if log== True:
                    print('\n====================================================\n',x0,'\n=============================================\n')
                
                if f_new <= f_old:
                    x0 = xnew
                    f_old = f_new
                    break
                else:
                    i+=1
        
        alpha = alpha/2
    
    return x0

## <center> Simulated Annealing

In [ ]:
def new_random(size,beta_max,alpha_max,max_iter,Q_max):
    """
        generates new random configuration
    """
    
    c = []

    #evaporation#
    p = np.random.uniform(0,1)
    c.append(p)

    #alpha#
    alpha = np.random.uniform(0,alpha_max)
    c.append(alpha)

    #beta#
    beta = np.random.uniform(0,beta_max)
    c.append(beta)

    #iterations#
    n = round(np.random.uniform(1,max_iter))
    c.append(n)

    #agents#
    k = round(np.random.uniform(1,size))
    c.append(k)

    #Q value#
    Q = np.random.uniform(0,Q_max)
    c.append(Q)

    return c    

In [ ]:
def get_cost(A,c,rep,size,beta_max,alpha_max,max_iter,Q_max):

    current_cost = 0

    for i in range(0,rep,1):
        temp_cost,_ = ACO(A,p=c[0],alpha=c[1],beta=c[2],n=c[3],k=c[4],Q=c[5],random_loc=True,update="best",ratio=0,max_rep=20,tol=0.001,log=False)
        current_cost+=temp_cost
    
    current_cost /=rep

    return current_cost
    

In [ ]:
def SA(A,c0,rep,Tmax,Tmin,epoch,size,beta_max,alpha_max,max_iter,Q_max,log):
    """
        finds the global optimal configuration using the SA algorithm

        Inputs:
            A         : the distance matrix
            c0        : initial Configuration of the probelm
            Tmax      : maximum temperature
            Tmin      : minimum temperature
            epoch     : number of repetitions
            size      : number of nodes
            beta_max  : maximum allowed beta value
            alpha_max : maximum allowed alpha value
            max_iter  : maximum allowed iterations
            Q_max     : 
    """

    c = c0
    current_cost = get_cost(A, c, rep, size, beta_max, alpha_max, max_iter, Q_max)

    for T in range(Tmax,Tmin,-1):

        for i in range(0,epoch,1):

            #current_cost = get_cost(A, c, rep, size, beta_max, alpha_max, max_iter, Q_max)
            c_new = new_random(size,beta_max,alpha_max,max_iter,Q_max)
            new_cost = get_cost(A, c_new, rep, size, beta_max, alpha_max, max_iter, Q_max)
            delta_c = (-1*new_cost) - (-1*current_cost)

            p = np.random.uniform(0,1)

            if delta_c>0:
                c = c_new
                current_cost=new_cost
            elif (np.exp(delta_c/T)>p):
                c = c_new
                current_cost=new_cost
    
        if log==True:
            print("Temperature: {}".format(T),"\n=================================\n","current config:\n",c,"\n cost: {}\n".format(current_cost),"\n=================================\n")

    return c

## <center> Symmetric Tests </center>

### Test 1

In [26]:
A = np.array([[np.inf,5,2,200],[5,np.inf,100,4],[2,100,np.inf,3],[200,4,3,np.inf]])
p = 0.5
alpha = 1
beta = 4
n = 100
k = 3
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='elite',ratio = 0.3,max_rep=40,tol = 0.001,log = False)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[0, 1, 3, 2, 0]

path distance was: 14.0




In [27]:
#Hyperparameter#
x0 = [0.5,1,4,100,3,1]
param = SA(A,x0,rep=5,Tmax=3000,Tmin=1,epoch=10,size=4,beta_max=20,alpha_max=20,max_iter=100,Q_max=20,log=True)


Temperature: 3000 
 current config:
 [0.6357644681178113, 0.25394694063017864, 7.875231396239684, 64, 3, 11.330100447115374] 
 cost: 14.0
 

Temperature: 2999 
 current config:
 [0.24401675421392377, 12.32175920659362, 1.5891467596237696, 66, 1, 14.579493221388717] 
 cost: 14.0
 

Temperature: 2998 
 current config:
 [0.8991116682982999, 3.919928213535875, 7.132734579103834, 89, 3, 2.128183533353216] 
 cost: 14.0
 

Temperature: 2997 
 current config:
 [0.4092914397865668, 17.431212251198026, 0.08283692688115352, 70, 3, 3.9126514843740234] 
 cost: 14.0
 

Temperature: 2996 
 current config:
 [0.46483932163497843, 15.082838586120637, 12.273072259976152, 26, 4, 15.173546443224186] 
 cost: 14.0
 

Temperature: 2995 
 current config:
 [0.45131391361265116, 12.139603527826544, 6.383205014233317, 2, 3, 17.83391745591042] 
 cost: 14.0
 

Temperature: 2994 
 current config:
 [0.8474474404350062, 8.279969005544183, 6.958476383045486, 41, 4, 3.6760325168089603] 
 cost: 14.0
 

Temperature: 2993 

KeyboardInterrupt: 

### Test 2

In [29]:
#TEST 2#
path = 'Data/st70/st70_tsp.txt'
A = get_distance_matrix_symmetric(path)
p = 0.011708620754442345
alpha = 12.238613702726342
beta = 10.05464961884304
n = 849
k = 26
Q = 9.52874757970913
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=False,update='best',ratio =0,max_rep=20,tol = 0.001,log = False)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[0, 35, 28, 12, 30, 68, 37, 58, 62, 21, 65, 52, 4, 9, 51, 59, 11, 20, 33, 32, 53, 61, 60, 39, 44, 38, 24, 45, 26, 8, 16, 42, 40, 5, 41, 17, 3, 1, 6, 31, 2, 43, 67, 29, 19, 13, 27, 7, 25, 48, 54, 18, 23, 14, 56, 34, 69, 22, 46, 36, 57, 49, 50, 55, 66, 47, 10, 64, 63, 15, 0]

path distance was: 728.3538117417551




In [26]:
#Hyperparameter#
path = 'Data/st70/st70_tsp.txt'
A = get_distance_matrix_symmetric(path)
x0 = [0.62,20,10,50,45,1]
param = SA(A,x0,rep=5,Tmax=3000,Tmin=1,epoch=1,size=70,beta_max=20,alpha_max=20,max_iter=10,Q_max=20,log=True)

KeyboardInterrupt: 

### Test 3

In [29]:
#TEST 3#
path = 'Data/eil101/eil101_tsp.txt'
A = get_distance_matrix_symmetric(path)
p = 0.6
alpha = 20
beta = 10
n = 700
k = 45
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='best',max_rep=20,tol = 0.001,log = True)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')

TypeError: ACO() missing 1 required positional argument: 'ratio'

### Test 4

In [ ]:
#TEST 4#
path = 'Data/a280/a280_tsp.txt'
A = get_distance_matrix_symmetric(path)
p = 0.6
alpha = 20
beta = 10
n = 100
k = 200
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='best',max_rep=20,tol = 0.001,log = True)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')

## <center> Asymmetric Tests </center>

### Test 1

In [30]:
A = np.array([(np.inf,13,5,2),(1,np.inf,7,6),(3,7,np.inf,1),(6,1,7,np.inf)])
p = 0.5
alpha = 1
beta = 4
n = 100
k = 3
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='best',ratio = 0,max_rep=20,tol = 0.001,log = False)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[0, 2, 3, 1, 0]

path distance was: 8.0




## test 2

In [64]:
path = 'Data/br17/br17.atsp'
probelm = tsplib95.load(path)
graph = probelm.get_graph()


A = networkx.to_numpy_matrix(graph)
A = np.squeeze(np.asarray(A))

m = len(A)
A[range(m),range(m)] = np.inf

p = 0.6
alpha = 20
beta = 10
n = 500
k = 17
Q = 1
dist,path = ACO(A,p,alpha,beta,n,k,Q,random_loc=True,update='best',ratio = 0,max_rep=20,tol = 0.001,log = False)

print('\n============================================\n')
print('path taken \n')
print(path)
print('\npath distance was:',dist)
print('\n============================================\n')



path taken 

[15, 4, 5, 3, 14, 12, 2, 10, 13, 1, 16, 9, 11, 8, 0, 7, 6, 15]

path distance was: 80.0




In [92]:
np.random.uniform(0,100)

48.49658582130479